In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder
from kmodes.kprototypes import KPrototypes
from matplotlib import style
from sklearn.preprocessing import MinMaxScaler

Import data

In [71]:
data = pd.read_csv('cleaned.csv')

C:\Users\G\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (11,18,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Copy the data in dataframe df

In [72]:
df=data.copy()
#df.head()

Keep data only for the year 2008 and 2009 (study period).

In [73]:
start=datetime.datetime(2008, 1, 1)
end=datetime.datetime(2009, 12, 31)

In [74]:
df['APPL_DATE']= pd.to_datetime(df['APPL_DATE'],dayfirst=True)
indices=[]

df = df[df['APPL_DATE']>start]
df = df[df['APPL_DATE']<end]


#for i in range(len(df)):
#    if i%100000==0:
#        print(i)
#    if df.iloc[i]['APPL_DATE']<start or df.iloc[i]['APPL_DATE']>end:
#        indices.append(i)
#df.drop(df.index[indices],inplace=True)

In [75]:
len(df)

239864

Remove columns which we don't want our model to take

In [76]:
df.drop(['OD_AMT','ACTIVITY_NAME','CLIENT_SRLNO','UNIT_NAME','APPL_DATE','PRODUCT_NAME','PIN_CODE','REGIS_DATE','ASSET_BASE','VILLAGE_NAME','REPAY_TYPE','Unnamed: 0','FAMILY_MEN','FAMILY_WOM','NO_OF_INSTALL','CITY','ACTIVE_YN','TOTAL_INSTALLMENT_PAID'],axis=1,inplace=True)
#len(df)
#df.drop(['TOTAL_INSTALLMENT_PAID'],axis=1,inplace=True)
#df.drop(['PIN_CODE','REGIS_DATE','ASSET_BASE'],axis=1,inplace=True)
#df.isnull().sum()*100/df.shape[0]

Selecting only the required district

In [77]:
df = df[df['UNIT_CODE']==31]

In [78]:
len(df)

5713

Remove NaN's and convert values in the data to str or int

In [79]:
df.dropna(inplace=True)
df = df.astype({"VILLAGE_CODE": str,'FAMILY_MEM':int,'GUARANTOR':str,'CO_OBLIGANT':str,'CO_LATERAL':str,'CATEGORY':str,'GENDER':str})
#len(df)
#df.head()

In [80]:
#for i in range(len(df)):
#    if i%100000==0:
 #       print(i)
  #  if df.iloc[i]['ODDAYS']!=0:
  #      df.iloc[i]['ODDAYS']=1
#df.loc[df['ODDAYS'] > 0, 'First Season'] = 1
df['Default'] = (df['ODDAYS'] > 0).astype(int)

In [81]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,ODDAYS,Default
1865,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0.0,0
1866,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,156.0,1
1867,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,156.0,1
1868,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,156.0,1
1869,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,156.0,1


In [82]:
df.drop('ODDAYS',inplace=True,axis=1)

Encode the categorical variables. 
Firct convert the dataframe in a numpy array and then choose the columns to encode.

In [83]:
labelencoder1=LabelEncoder()

In [84]:
X=df.iloc[:,1:19].values
#X.shape

In [85]:
df.reset_index(inplace=True)

In [86]:
df.drop('index',axis=1,inplace=True)
#print(df.head())

In [87]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1


In [88]:
for i in range (0,17):
    if i!=3 or i!=6 or i!=8 or i!=9 or i!=15:
        X[:,i]=labelencoder1.fit_transform(X[:,i])
        #print (i)

In [89]:
df.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1


In [90]:
df2=df.copy()

Scaling of numerical variables.

In [91]:
scaler = MinMaxScaler()
df[['APPL_AMT', 'FAMILY_MEM','REGULAR','FESTIVE','AGE']] = scaler.fit_transform(df[['APPL_AMT', 'FAMILY_MEM','REGULAR','FESTIVE','AGE']])

C:\Users\G\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


ID is the identifier for each entry in out data

In [92]:
ID = df['ID_NO'].to_list()
ID= np.asarray(ID)
#len(ID)

In [93]:
#style.use("ggplot")
#colors = ['b', 'orange', 'g', 'r', 'c', 'm', 'y', 'k', 'Brown', 'ForestGreen']

In [94]:
print(X.shape)

(4189, 18)


Fitting the model k-prototype.
n_init = number of runs, 
categorical = column numbers of categorical variables, 
X = numpy array formed from the original data leaving the identifier column ID out, 
n_clusters = number of clusters

In [95]:
n_clust=10
kproto = KPrototypes(n_clusters=n_clust, init='Cao', verbose=2,n_init=5)
clusters = kproto.fit_predict(X, categorical=[0,1,3,4,6,9,10,11,12,13,15,16])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 1193, ncost: 251036.50345945277
Run: 1, iteration: 2/100, moves: 566, ncost: 242092.58021580207
Run: 1, iteration: 3/100, moves: 427, ncost: 236698.2329477677
Run: 1, iteration: 4/100, moves: 257, ncost: 234784.29382014266
Run: 1, iteration: 5/100, moves: 300, ncost: 230185.6364865568
Run: 1, iteration: 6/100, moves: 261, ncost: 226683.2777074381
Run: 1, iteration: 7/100, moves: 161, ncost: 224934.97482407474
Run: 1, iteration: 8/100, moves: 85, ncost: 224650.49156585272
Run: 1, iteration: 9/100, moves: 23, ncost: 224603.80074043857
Run: 1, iteration: 10/100, moves: 7, ncost: 224599.28587902893
Run: 1, iteration: 11/100, moves: 1, ncost: 224598.89831443736
Run: 1, iteration: 12/100, moves: 0, ncost: 224598.89831443736
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 1134, ncost: 235001.36821314073
Run: 2, iterat

Location of the centroids

In [96]:
#print(kproto.cluster_centroids_)

Save the obtained clusters in a dictionary where keys are ID's and values are the cluster numbers. Also, append the cluster numbers to original dataframe.

In [97]:
clust_dict={}
clust=[]
for s, c in zip(ID, clusters):
    clust_dict[s]=c
    clust.append(c)
df2['cluster']=clust

In [98]:
len(clust)

4189

In [99]:
df2.to_csv('cleaned_with_clusters_vs_ID_Adilabad.csv')

In [100]:
df2.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,8
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,9
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,0


In [101]:
for i in range(0,n_clust):
    print ("Cluster",i,":", df2.groupby('cluster').count().iloc[i]['ID_NO'])
#df.groupby('cluster').count()

Cluster 0 : 624
Cluster 1 : 505
Cluster 2 : 304
Cluster 3 : 432
Cluster 4 : 232
Cluster 5 : 116
Cluster 6 : 379
Cluster 7 : 715
Cluster 8 : 380
Cluster 9 : 502


In [102]:
summary = pd.read_csv('cleaned_with_clusters_vs_ID_Adilabad.csv')

In [103]:
summary.groupby('cluster').count()

,Unnamed: 0,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default
cluster,,,,,,,,,,,,,,,,,,,,
0,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624,624
1,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505,505
2,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304,304
3,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432,432
4,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232,232
5,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116,116
6,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379,379
7,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715,715
8,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380,380


In [104]:
summary.drop('Unnamed: 0',inplace=True,axis=1)

Based on category

In [105]:
print('Clusterwise category for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('CATEGORY').count()
df1.reset_index(inplace=True)
#df1.head()
df1 = df1[['CATEGORY','ID_NO']].copy()

print('Category-wise summary for all clusters:\n')
for cat in summary['CATEGORY'].unique().tolist():
    print('\nFor category',cat,'the total number of applications are-',df1[df1['CATEGORY']==cat]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].tolist())
        cats = summary[summary['cluster']==i]['CATEGORY'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'CAT':cats,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['CAT']==cat]['count']*100/float(df1[df1['CATEGORY']==cat]['ID_NO'].to_string(index=None)))
            

Clusterwise category for all clusters

Cluster 0 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[53.53, 19.07, 17.47, 8.33, 1.6]
Cluster 1 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[51.49, 21.98, 16.63, 7.92, 1.98]
Cluster 2 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[48.36, 29.28, 13.82, 7.24, 1.32]
Cluster 3 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[58.1, 18.29, 15.74, 5.79, 2.08]
Cluster 4 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[48.71, 26.72, 13.79, 8.19, 2.59]
Cluster 5 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[41.38, 28.45, 12.93, 12.93, 4.31]
Cluster 6 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[53.03, 19.53, 19.26, 7.12, 1.06]
Cluster 7 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[54.83, 21.26, 16.78, 6.15, 0.98]
Cluster 8 :-
['O.B.C', 'S.C', 'MINORITY', 'S.T', 'O.C']
[45.26, 23.16, 22.11, 8.42, 1.05]
Cluster 9 :-
['O.B.C', 'MINORITY', 'S.C', 'S.T', 'O.C']
[48.21, 21.91, 19.92, 8.57, 1.39]
Category-wise summary for all clusters:


For category O.B.C t

Based on default behaviour

In [106]:
print('Clusterwise defualt behaviour\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist())
        print(count)
        
df1 = summary.groupby('Default').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['Default','ID_NO']].copy()

print('Default behaviour wise summary for all clusters:\n')
for default in summary['Default'].unique().tolist():
    print('\nFor default',default,'the total number of applications are-',df1[df1['Default']==default]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['Default'].value_counts()[:2].tolist())
        defs = summary[summary['cluster']==i]['Default'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Def':defs,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Def']==default]['count']*100/float(df1[df1['Default']==default]['ID_NO'].to_string(index=None)))
            

Clusterwise defualt behaviour

Cluster 0 :-
[1, 0]
[53.21, 46.79]
Cluster 1 :-
[0, 1]
[59.21, 40.79]
Cluster 2 :-
[0, 1]
[65.46, 34.54]
Cluster 3 :-
[0, 1]
[54.63, 45.37]
Cluster 4 :-
[0, 1]
[70.69, 29.31]
Cluster 5 :-
[1, 0]
[51.72, 48.28]
Cluster 6 :-
[0, 1]
[65.44, 34.56]
Cluster 7 :-
[0, 1]
[50.77, 49.23]
Cluster 8 :-
[0, 1]
[60.0, 40.0]
Cluster 9 :-
[0, 1]
[61.35, 38.65]
Default behaviour wise summary for all clusters:


For default 0 the total number of applications are-  2393
Cluster 0 has below %
1    12.202257
Name: count, dtype: float64
Cluster 1 has below %
0    12.494776
Name: count, dtype: float64
Cluster 2 has below %
0    8.315921
Name: count, dtype: float64
Cluster 3 has below %
0    9.862098
Name: count, dtype: float64
Cluster 4 has below %
0    6.853322
Name: count, dtype: float64
Cluster 5 has below %
1    2.340159
Name: count, dtype: float64
Cluster 6 has below %
0    10.36356
Name: count, dtype: float64
Cluster 7 has below %
0    15.169244
Name: count, dtype: float

Based on Secured/Unsecured status

In [107]:
print('Clusterwise secured/unsecured for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:5].index.tolist())
        print(count)
        
df1 = summary.groupby('UNSECURE_SECURE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['UNSECURE_SECURE','ID_NO']].copy()


print('Secured/unsecured wise summary for all clusters:\n')
for status in summary['UNSECURE_SECURE'].unique().tolist():
    print('\nFor status',status,'the total number of applications are-',df1[df1['UNSECURE_SECURE']==status]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].tolist())
        statuses = summary[summary['cluster']==i]['UNSECURE_SECURE'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'STATUS':statuses,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['STATUS']==status]['count']*100/float(df1[df1['UNSECURE_SECURE']==status]['ID_NO'].to_string(index=None)))
            

Clusterwise secured/unsecured for all clusters

Cluster 0 :-
['U']
[100.0]
Cluster 1 :-
['U']
[100.0]
Cluster 2 :-
['U']
[100.0]
Cluster 3 :-
['U']
[100.0]
Cluster 4 :-
['U']
[100.0]
Cluster 5 :-
['U']
[100.0]
Cluster 6 :-
['U']
[100.0]
Cluster 7 :-
['U']
[100.0]
Cluster 8 :-
['U']
[100.0]
Cluster 9 :-
['U']
[100.0]
Secured/unsecured wise summary for all clusters:


For status U the total number of applications are-  4189
Cluster 0 has below %
0    14.896157
Name: count, dtype: float64
Cluster 1 has below %
0    12.055383
Name: count, dtype: float64
Cluster 2 has below %
0    7.257102
Name: count, dtype: float64
Cluster 3 has below %
0    10.312724
Name: count, dtype: float64
Cluster 4 has below %
0    5.538315
Name: count, dtype: float64
Cluster 5 has below %
0    2.769157
Name: count, dtype: float64
Cluster 6 has below %
0    9.047505
Name: count, dtype: float64
Cluster 7 has below %
0    17.068513
Name: count, dtype: float64
Cluster 8 has below %
0    9.071377
Name: count, dtype: fl

In [108]:
#Based on house type
print('Clusterwise house type for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist())
        print(count)        
        
df1 = summary.groupby('HOUSE_TYPE').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['HOUSE_TYPE','ID_NO']].copy()


print('House type wise summary for all clusters:\n')
for htype in summary['HOUSE_TYPE'].unique().tolist():
    print('\nFor house type',htype,'the total number of applications are-',df1[df1['HOUSE_TYPE']==htype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].tolist())
        types = summary[summary['cluster']==i]['HOUSE_TYPE'].value_counts()[:5].index.tolist()
        df_dummy = pd.DataFrame({'HOUSE':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['HOUSE']==htype]['count']*100/float(df1[df1['HOUSE_TYPE']==htype]['ID_NO'].to_string(index=None)))
            

Clusterwise house type for all clusters

Cluster 0 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C']
[66.51, 32.53, 0.64, 0.32]
Cluster 1 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[63.17, 31.09, 2.97, 2.57, 0.2]
Cluster 2 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[60.2, 34.87, 3.62, 1.32]
Cluster 3 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[66.9, 31.94, 0.69, 0.46]
Cluster 4 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[70.26, 24.14, 3.88, 1.72]
Cluster 5 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[68.97, 28.45, 1.72, 0.86]
Cluster 6 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets']
[56.46, 38.79, 2.64, 2.11]
Cluster 7 :-
['Tatched', 'Tiles', 'R.C.C', 'Sheets', 'Stones']
[64.34, 33.57, 1.26, 0.7, 0.14]
Cluster 8 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C', 'Stones']
[59.74, 32.37, 3.95, 3.42, 0.53]
Cluster 9 :-
['Tatched', 'Tiles', 'Sheets', 'R.C.C', 'Stones']
[56.37, 30.68, 6.37, 6.18, 0.4]
House type wise summary for all clusters:


For house type Tiles the total number of applications ar

Based on Guarantor

In [109]:
print('Clusterwise guarantor for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist())
        print(count)        
        
df1 = summary.groupby('GUARANTOR').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GUARANTOR','ID_NO']].copy()


print('Gurantor based summary for all clusters:\n')
for gtype in summary['GUARANTOR'].unique().tolist():
    print('\nFor guarantor',gtype,'the total number of applications are-',df1[df1['GUARANTOR']==htype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GUARANTOR'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Guarantor':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Guarantor']==gtype]['count']*100/float(df1[df1['GUARANTOR']==gtype]['ID_NO'].to_string(index=None)))
            

Clusterwise guarantor for all clusters

Cluster 0 :-
['N']
[100.0]
Cluster 1 :-
['N']
[100.0]
Cluster 2 :-
['N']
[100.0]
Cluster 3 :-
['N']
[100.0]
Cluster 4 :-
['N']
[100.0]
Cluster 5 :-
['N']
[100.0]
Cluster 6 :-
['N']
[100.0]
Cluster 7 :-
['N']
[100.0]
Cluster 8 :-
['N']
[100.0]
Cluster 9 :-
['N']
[100.0]
Gurantor based summary for all clusters:


For guarantor N the total number of applications are- Series([], )
Cluster 0 has below %
0    14.896157
Name: count, dtype: float64
Cluster 1 has below %
0    12.055383
Name: count, dtype: float64
Cluster 2 has below %
0    7.257102
Name: count, dtype: float64
Cluster 3 has below %
0    10.312724
Name: count, dtype: float64
Cluster 4 has below %
0    5.538315
Name: count, dtype: float64
Cluster 5 has below %
0    2.769157
Name: count, dtype: float64
Cluster 6 has below %
0    9.047505
Name: count, dtype: float64
Cluster 7 has below %
0    17.068513
Name: count, dtype: float64
Cluster 8 has below %
0    9.071377
Name: count, dtype: float64


In [110]:
#Based on Co obligant

print('Clusterwise co obligant for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_OBLIGANT').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_OBLIGANT','ID_NO']].copy()


print('Guarantor based summary for all clusters:\n')
for otype in summary['CO_OBLIGANT'].unique().tolist():
    print('\nFor co obligant',otype,'the total number of applications are-',df1[df1['CO_OBLIGANT']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_OBLIGANT'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Obligant':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Obligant']==otype]['count']*100/float(df1[df1['CO_OBLIGANT']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise co obligant for all clusters

Cluster 0 :-
['N', 'O']
[98.56, 1.44]
Cluster 1 :-
['N', 'O']
[95.25, 4.75]
Cluster 2 :-
['N', 'O']
[96.71, 3.29]
Cluster 3 :-
['N', 'O']
[99.31, 0.69]
Cluster 4 :-
['N', 'O']
[98.71, 1.29]
Cluster 5 :-
['N', 'O']
[97.41, 2.59]
Cluster 6 :-
['N', 'O']
[96.57, 3.43]
Cluster 7 :-
['N', 'O']
[97.34, 2.66]
Cluster 8 :-
['N', 'O']
[97.37, 2.63]
Cluster 9 :-
['N', 'O']
[98.61, 1.39]
Guarantor based summary for all clusters:


For co obligant N the total number of applications are-  4088
Cluster 0 has below %
0    15.044031
Name: count, dtype: float64
Cluster 1 has below %
0    11.766145
Name: count, dtype: float64
Cluster 2 has below %
0    7.191781
Name: count, dtype: float64
Cluster 3 has below %
0    10.494129
Name: count, dtype: float64
Cluster 4 has below %
0    5.601761
Name: count, dtype: float64
Cluster 5 has below %
0    2.764188
Name: count, dtype: float64
Cluster 6 has below %
0    8.953033
Name: count, dtype: float64
Cluster 7 has below %

In [111]:
#Based on Collateral

print('Clusterwise collateral for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('CO_LATERAL').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['CO_LATERAL','ID_NO']].copy()


print('\nCollateral based summary for all clusters:')
for otype in summary['CO_LATERAL'].unique().tolist():
    print('\nFor collateral',otype,'the total number of applications are-',df1[df1['CO_LATERAL']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['CO_LATERAL'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'Collateral':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['Collateral']==otype]['count']*100/float(df1[df1['CO_LATERAL']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise collateral for all clusters

Cluster 0 :-
['N']
[100.0]
Cluster 1 :-
['N']
[100.0]
Cluster 2 :-
['N']
[100.0]
Cluster 3 :-
['N']
[100.0]
Cluster 4 :-
['N']
[100.0]
Cluster 5 :-
['N']
[100.0]
Cluster 6 :-
['N']
[100.0]
Cluster 7 :-
['N']
[100.0]
Cluster 8 :-
['N']
[100.0]
Cluster 9 :-
['N']
[100.0]

Collateral based summary for all clusters:

For collateral N the total number of applications are-  4189
Cluster 0 has below %
0    14.896157
Name: count, dtype: float64
Cluster 1 has below %
0    12.055383
Name: count, dtype: float64
Cluster 2 has below %
0    7.257102
Name: count, dtype: float64
Cluster 3 has below %
0    10.312724
Name: count, dtype: float64
Cluster 4 has below %
0    5.538315
Name: count, dtype: float64
Cluster 5 has below %
0    2.769157
Name: count, dtype: float64
Cluster 6 has below %
0    9.047505
Name: count, dtype: float64
Cluster 7 has below %
0    17.068513
Name: count, dtype: float64
Cluster 8 has below %
0    9.071377
Name: count, dtype: float64
Clu

In [112]:
#Based on Gender

print('Clusterwise gender for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('GENDER').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['GENDER','ID_NO']].copy()


print('\nGender based summary for all clusters:')
for otype in summary['GENDER'].unique().tolist():
    print('\nFor gender',otype,'the total number of applications are-',df1[df1['GENDER']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['GENDER'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['GENDER'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'gender':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['gender']==otype]['count']*100/float(df1[df1['GENDER']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise gender for all clusters

Cluster 0 :-
['F', 'M']
[65.38, 34.62]
Cluster 1 :-
['F', 'M']
[60.2, 39.8]
Cluster 2 :-
['F', 'M']
[53.29, 46.71]
Cluster 3 :-
['F', 'M']
[66.2, 33.8]
Cluster 4 :-
['F', 'M']
[55.6, 44.4]
Cluster 5 :-
['F', 'M']
[72.41, 27.59]
Cluster 6 :-
['F', 'M']
[53.56, 46.44]
Cluster 7 :-
['F', 'M']
[52.31, 47.69]
Cluster 8 :-
['F', 'M']
[72.11, 27.89]
Cluster 9 :-
['F', 'M']
[74.7, 25.3]

Gender based summary for all clusters:

For gender F the total number of applications are-  2599
Cluster 0 has below %
0    15.698346
Name: count, dtype: float64
Cluster 1 has below %
0    11.696806
Name: count, dtype: float64
Cluster 2 has below %
0    6.233167
Name: count, dtype: float64
Cluster 3 has below %
0    11.004232
Name: count, dtype: float64
Cluster 4 has below %
0    4.963447
Name: count, dtype: float64
Cluster 5 has below %
0    3.232012
Name: count, dtype: float64
Cluster 6 has below %
0    7.810696
Name: count, dtype: float64
Cluster 7 has below %
0    14.39

In [113]:
summary.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,8
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,9
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,0


In [114]:
#Based on Accomodation

print('Clusterwise accomodation for all clusters\n')
for i in range(0,n_clust):
        print('Cluster',i,':-')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        total_in_cluster = summary.groupby('cluster').count().iloc[i]['ID_NO']
        count = [((x*100)/total_in_cluster).round(2) for x in count]
        print(summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist())
        print(count)        

df1 = summary.groupby('ACCOMADATION').count()
df1.reset_index(inplace=True)
df1.head()
df1 = df1[['ACCOMADATION','ID_NO']].copy()


print('\nAccomodation based summary for all clusters:')
for otype in summary['ACCOMADATION'].unique().tolist():
    print('\nFor accomodation',otype,'the total number of applications are-',df1[df1['ACCOMADATION']==otype]['ID_NO'].to_string(index=None))
    for i in range(0,n_clust):
        print('Cluster',i,'has below %')
        count = (summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].tolist())
        types = summary[summary['cluster']==i]['ACCOMADATION'].value_counts()[:2].index.tolist()
        df_dummy = pd.DataFrame({'acco':types,'count':count})
        #print(df_dummy.head())
        print(df_dummy[df_dummy['acco']==otype]['count']*100/float(df1[df1['ACCOMADATION']==otype]['ID_NO'].to_string(index=None)))
            

Clusterwise accomodation for all clusters

Cluster 0 :-
['O', 'R']
[70.19, 29.81]
Cluster 1 :-
['R', 'O']
[90.89, 9.11]
Cluster 2 :-
['R', 'O']
[67.11, 32.89]
Cluster 3 :-
['O', 'R']
[53.47, 46.53]
Cluster 4 :-
['R', 'O']
[70.69, 29.31]
Cluster 5 :-
['O', 'R']
[79.31, 20.69]
Cluster 6 :-
['R', 'O']
[97.63, 2.37]
Cluster 7 :-
['O', 'R']
[56.08, 43.92]
Cluster 8 :-
['R', 'O']
[90.53, 9.47]
Cluster 9 :-
['R', 'O']
[88.05, 11.95]

Accomodation based summary for all clusters:

For accomodation R the total number of applications are-  2708
Cluster 0 has below %
1    6.868538
Name: count, dtype: float64
Cluster 1 has below %
0    16.949778
Name: count, dtype: float64
Cluster 2 has below %
0    7.533235
Name: count, dtype: float64
Cluster 3 has below %
1    7.422452
Name: count, dtype: float64
Cluster 4 has below %
0    6.05613
Name: count, dtype: float64
Cluster 5 has below %
1    0.886263
Name: count, dtype: float64
Cluster 6 has below %
0    13.66322
Name: count, dtype: float64
Cluster 7 ha

SUMMARY STATS FOR CONTINUOUS NUMERICAL VARIABLES: APPL_AMT, FAMILY_MEM, REGULAR, FESTIVE, AGE

In [115]:
summary.head()

,ID_NO,UNIT_CODE,PRODUCT_CODE,APPL_AMT,ACTIVITY_CODE,VILLAGE_CODE,FAMILY_MEM,HOUSE_TYPE,REGULAR,FESTIVE,UNSECURE_SECURE,GUARANTOR,CO_OBLIGANT,CO_LATERAL,CATEGORY,AGE,GENDER,ACCOMADATION,Default,cluster
0,8749.0,31,113.0,10000.0,81.0,A0006,4,Tiles,1000.0,5000.0,U,N,N,N,O.B.C,32.0,F,R,0,2
1,8787.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,27.0,F,R,1,1
2,8788.0,31,103.0,20000.0,22.0,UT001,6,Tiles,1000.0,2000.0,U,N,N,N,O.B.C,41.0,F,R,1,8
3,8794.0,31,104.0,20000.0,33.0,UT001,6,Tatched,1000.0,2000.0,U,N,N,N,O.B.C,31.0,F,R,1,9
4,8795.0,31,103.0,20000.0,22.0,UT001,3,Tiles,500.0,1000.0,U,N,N,N,O.B.C,32.0,F,R,1,0


In [116]:
for i in range(0,n_clust):
    sliced = summary[summary['cluster']==i]
    print('Stats for cluster',i,':-')
    print('1. Application amount-','Mean:',round(sliced['APPL_AMT'].mean(),ndigits=2),',Min:',round(sliced['APPL_AMT'].min(),ndigits=2),',Max:',round(sliced['APPL_AMT'].max(),ndigits=2),',Median:',round(sliced['APPL_AMT'].median(),ndigits=2))
    print('2. Family members-','Mean:',round(sliced['FAMILY_MEM'].mean(),ndigits=2),',Min:',round(sliced['FAMILY_MEM'].min(),ndigits=2),',Max:',round(sliced['FAMILY_MEM'].max(),ndigits=2),',Median:',round(sliced['FAMILY_MEM'].median(),ndigits=2))
    print('3. Regular-','Mean:',round(sliced['REGULAR'].mean(),ndigits=2),',Min:',round(sliced['REGULAR'].min(),ndigits=2),',Max:',round(sliced['REGULAR'].max(),ndigits=2),',Median:',round(sliced['REGULAR'].median(),ndigits=2))
    print('4. Festive-','Mean:',round(sliced['FESTIVE'].mean(),ndigits=2),',Min:',round(sliced['FESTIVE'].min(),ndigits=2),',Max:',round(sliced['FESTIVE'].max(),ndigits=2),',Median:',round(sliced['FESTIVE'].median(),ndigits=2))
    print('5. Age-','Mean:',round(sliced['AGE'].mean(),ndigits=2),',Min:',round(sliced['AGE'].min(),ndigits=2),',Max:',round(sliced['AGE'].max(),ndigits=2),',Median:',round(sliced['AGE'].median(),ndigits=2))
    print('\n')

Stats for cluster 0 :-
1. Application amount- Mean: 12439.9 ,Min: 8000.0 ,Max: 50000.0 ,Median: 12000.0
2. Family members- Mean: 2.57 ,Min: 1 ,Max: 7 ,Median: 2.0
3. Regular- Mean: 492.34 ,Min: 50.0 ,Max: 800.0 ,Median: 500.0
4. Festive- Mean: 890.29 ,Min: 90.0 ,Max: 2000.0 ,Median: 900.0
5. Age- Mean: 34.68 ,Min: 30.0 ,Max: 40.0 ,Median: 34.0


Stats for cluster 1 :-
1. Application amount- Mean: 14231.58 ,Min: 5000.0 ,Max: 80000.0 ,Median: 15000.0
2. Family members- Mean: 3.39 ,Min: 1 ,Max: 9 ,Median: 3.0
3. Regular- Mean: 999.5 ,Min: 450.0 ,Max: 3000.0 ,Median: 1000.0
4. Festive- Mean: 1846.18 ,Min: 1000.0 ,Max: 3500.0 ,Median: 2000.0
5. Age- Mean: 24.86 ,Min: 18.0 ,Max: 29.0 ,Median: 25.0


Stats for cluster 2 :-
1. Application amount- Mean: 14029.61 ,Min: 5000.0 ,Max: 150000.0 ,Median: 15000.0
2. Family members- Mean: 3.94 ,Min: 1 ,Max: 10 ,Median: 4.0
3. Regular- Mean: 2059.22 ,Min: 800.0 ,Max: 10002.0 ,Median: 2000.0
4. Festive- Mean: 4557.89 ,Min: 1500.0 ,Max: 20000.0 ,Median: 5